### Libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import pathlib
from tqdm import tqdm
import random
import pandas as pd
import os, sys, inspect
import torchvision as tv
import torchvision.models as models
import argparse
import time
from scipy.stats import binom
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import pickle as pkl
from scipy.optimize import brentq
from scipy.stats import binom, beta
from scipy.special import softmax
import pdb
import glob
import seaborn as sns
dirname = str(pathlib.Path().absolute())
sys.path.insert(1, os.path.join(sys.path[0], '../'))
from matplotlib.font_manager import FontProperties


### Helping function for PCOSQ

In [ ]:
def NoisyRC(range_bounds, D, sigma):
    """
    Noisy Range Count for float values with Gaussian noise.

    Parameters:
    range_bounds (tuple): A tuple (a, b) representing the range [a, b].
    D (list): The sorted dataset.
    sigma (float): The standard deviation of the Gaussian noise.

    Returns:
    int: The noisy count of elements in the range [a, b].
    """
    a, b = range_bounds
    count = sum(1 for z in D if a <= z <= b)
    noise = np.random.normal(0, sigma)
    noisy_count = count + noise
    return max(0, int(np.floor(noisy_count)))  # Ensure non-negative count

def PrivQuant(D, alpha, rho, seed, lower_bound=0, upper_bound=1, delta=1e-10):
    """
    Differentially Private Quantile Approximation Algorithm without integer conversion.

    Parameters:
    D (list): The sorted dataset.
    alpha (float): The quantile level (e.g., 0.5 for median).
    rho (float): The privacy parameter (smaller = more private).
    lower_bound (float): Lower bound of the search space.
    upper_bound (float): Upper bound of the search space.
    delta (float): Small positive value to ensure convergence.

    Returns:
    float: A differentially private approximation of the quantile x_{(m)}.
    """

    
    n = len(D)
    max_iterations = int(np.ceil(np.log2((upper_bound - lower_bound) / delta)))
    sigma = np.sqrt(max_iterations / (2 * rho)) # Noise scale for Gaussian mechanism
    m = int(np.ceil((1 - alpha) * (n + 1)))

    left, right = lower_bound, upper_bound
    random.seed(seed)
    for i in range(max_iterations):
        mid = (left + right) / 2
        c = NoisyRC((lower_bound, mid), D, sigma)
        
        if c < m:
            left = mid + delta
        else:
            right = mid

    return np.round((left + right) / 2, 2)



### Helping function Lap_hist

In [ ]:
def dp_quantile_noisy_hist(x, q, epsilon, seed, bins=50, domain=(0.0, 1.0), rng=None):
    """
    Differentially private quantile using a Laplace-noised histogram (ε-DP).

    Args:
        x (array-like): data vector (numeric).
        q (float): desired quantile in (0,1).
        epsilon (float): privacy budget for the entire histogram.
        domain (tuple): (lo, hi) public bounds for clipping/binning.
        bins (int): number of fixed, public bins.
        rng: np.random.Generator (optional).

    Returns:
        float: DP quantile estimate (can lie between data points).

    Privacy & assumptions:
        - Data are clipped to the public domain (lo, hi).
        - Build a fixed-bin histogram, add Lap(1/ε) noise to each bin count.
        - Because each record contributes to exactly one bin, releasing
          the full noisy histogram is ε-DP under add/remove adjacency.
        - Quantile is computed from the noisy cumulative counts.

    Notes:
        - Works best if a reasonable public domain is known.
        - For stability, negative noisy counts are floored at 0.
    """
    x = np.asarray(x, dtype=float)
    if x.size == 0:
        raise ValueError("x must be non-empty.")
    if not (0 < q < 1):
        raise ValueError("q must be in (0,1).")
    if epsilon <= 0:
        raise ValueError("epsilon must be > 0.")
    if rng is None:
        rng = np.random.default_rng(seed)

    lo, hi = domain
    if not (lo < hi):
        raise ValueError("domain must satisfy lo < hi.")

    # Clip to public domain
    xc = np.clip(x, lo, hi)

    # Fixed public bins
    edges = np.linspace(lo, hi, bins + 1)
    #print(f"Bins: {edges}")
    counts, _ = np.histogram(xc, bins=edges)
    #print(f"Counts of histogram: {counts}")

    # Laplace noise to each bin (scale = 1/ε)
    noise = rng.laplace(loc=0.0, scale=1.0/epsilon, size=bins)
    #print(f"Noise for each bin: {noise}")
    noisy = np.maximum(counts + noise, 0.0)
    #print(noisy)

    # Cumulative proportion
    csum = np.cumsum(noisy)
    if csum[-1] <= 0:
        # extremely unlikely unless ε is tiny and n is tiny
        return float(np.median(xc))

    target = q * csum[-1]
    j = np.searchsorted(csum, target)  # first bin reaching the target

    j = int(np.clip(j, 0, bins - 1))
    # Linear interpolation within the bin (simple, uniform-within-bin)
    bin_lo, bin_hi = edges[j], edges[j + 1]
    prev = csum[j - 1] if j > 0 else 0.0
    within = (target - prev) / max(noisy[j], 1e-12)
    within = np.clip(within, 0.0, 1.0)
    return float(bin_lo + within * (bin_hi - bin_lo))

### Helping function for EXPONQ

In [ ]:

def get_qtilde(n,alpha,gamma,epsilon,m):
    qtilde = (n+1)*(1-alpha)/(n*(1-gamma*alpha))+2/(epsilon*n)*np.log(m/(gamma*alpha))
    qtilde = min(qtilde, 1-1e-12)
    return qtilde

def generate_scores(n):
    return np.random.uniform(size=(n,))

def hist_2_cdf(cumsum, bins, n):
    def _cdf(t):
        if t > bins[-2]:
            return 1.0
        elif t < bins[1]:
            return 0.0
        else:
            return 1-cumsum[np.searchsorted(bins, t)]/n
    return _cdf




    



def get_private_quantile(scores, alpha, epsilon, gamma, bins):
    n = scores.shape[0]
    epsilon_normed = epsilon*min(alpha, 1-alpha)
    # Get the quantile
    qtilde = get_qtilde(n, alpha, gamma, epsilon, bins.shape[0])
    scores = scores.squeeze()
    score_to_bin = np.digitize(scores,bins)
    binned_scores = bins[np.minimum(score_to_bin,bins.shape[0]-1)]
    w1 = np.digitize(binned_scores, bins)
    w2 = np.digitize(binned_scores, bins, right=True)
    # Clip bins
    w1 = np.maximum(np.minimum(w1,bins.shape[0]-1),0)
    w2 = np.maximum(np.minimum(w2,bins.shape[0]-1),0)
    lower_mass = np.bincount(w1,minlength=bins.shape[0]).cumsum()/qtilde
    upper_mass = (n-np.bincount(w2,minlength=bins.shape[0]).cumsum())/(1-qtilde)
    w = np.maximum( lower_mass , upper_mass )
    sampling_probabilities = softmax(-(epsilon_normed/2)*w)
    # Check
    sampling_probabilities = sampling_probabilities/sampling_probabilities.sum()
    qhat = np.random.choice(bins,p=sampling_probabilities)
    return qhat

# Optimal gamma is a root.
def get_optimal_gamma(scores,n,alpha,m,epsilon):
    a = alpha**2
    b = - ( alpha*epsilon*(n+1)*(1-alpha)/2 + 2*alpha )
    c = 1
    best_q = 1
    gamma1 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
    gamma2 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

    gamma1 = min(max(gamma1,1e-12),1-1e-12)
    gamma2 = min(max(gamma2,1e-12),1-1e-12)

    bins = np.linspace(0,1,m)

    q1 = get_private_quantile(scores, alpha, epsilon, gamma1, bins)
    q2 = get_private_quantile(scores, alpha, epsilon, gamma2, bins)

    return (gamma1, q1) if q1 < q2 else (gamma2, q2)

def get_optimal_gamma_m(n, alpha, epsilon):
    candidates_m = np.logspace(4,6,50).astype(int)
    scores = np.random.rand(n,1)
    best_m = int(1/alpha)
    best_gamma = 1
    best_q = 1
    for m in candidates_m:
        gamma, q = get_optimal_gamma(scores,n,alpha,m,epsilon)
        if q < best_q:
            best_q = q
            best_m = m
            best_gamma = gamma
    return best_m, best_gamma




def get_conformal_scores(scores, labels):
    conformal_scores = torch.tensor([scores[i,labels[i]] for i in range(scores.shape[0])]) 
    return conformal_scores 

def get_shat_from_scores_private(scores, alpha, epsilon, gamma, score_bins):
    shat = get_private_quantile(scores, alpha, epsilon, gamma, score_bins)
    return shat

### Helping function for model training

In [ ]:

def get_model(modelname):
    """
    Load a pre-trained model by name.

    Args:
        modelname (str): Name of the model (e.g., 'ResNet152').

    Returns:
        torch.nn.Module: Pre-trained model.
    """
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    
    # Use the new weights parameter
    if modelname == 'ResNet18':
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    elif modelname == 'ResNet50':
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    elif modelname == 'ResNet101':
        model = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V1)
    elif modelname == 'ResNet152':
        model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
    elif modelname == 'ResNeXt101':
        model = models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.IMAGENET1K_V1)
    elif modelname == 'VGG16':
        model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    elif modelname == 'ShuffleNet':
        model = models.shufflenet_v2_x1_0(weights=models.ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1)
    elif modelname == 'Inception':
        model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
    elif modelname == 'DenseNet161':
        model = models.densenet161(weights=models.DenseNet161_Weights.IMAGENET1K_V1)
    else:
        raise NotImplementedError(f"Model {modelname} not supported")

    model.eval()
    model.to(device)
    return model

# Computes logits and targets from a model and loader
def get_logits_targets(model, loader):
    """
    Compute logits and targets for a dataset using a model.

    Args:
        model (torch.nn.Module): Pre-trained model.
        loader (torch.utils.data.DataLoader): DataLoader for the dataset.

    Returns:
        torch.Tensor: Tuple of (logits, targets).
    """
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    logits = torch.zeros((len(loader.dataset), 1000))  # 1000 classes in ImageNet
    labels = torch.zeros((len(loader.dataset),))
    i = 0

    print(f'Computing logits for model (only happens once).')
    with torch.no_grad():
        for x, targets in tqdm(loader, desc="Processing batches"):
            batch_logits = model(x.to(device)).detach().cpu()
            logits[i:(i + x.shape[0]), :] = batch_logits
            labels[i:(i + x.shape[0])] = targets.cpu()
            i += x.shape[0]

    # Construct the dataset
    dataset_logits = torch.utils.data.TensorDataset(logits, labels.long())
    return dataset_logits



def get_logits_dataset(modelname, datasetname, datasetpath, cache= dirname + '/.cache/'):
    fname = cache + datasetname + '/' + modelname + '.pkl' 

    # If the file exists, load and return it.
    if os.path.exists(fname):
        with open(fname, 'rb') as handle:
            return pkl.load(handle)

    # Else we will load our model, run it on the dataset, and save/return the output.
    model = get_model(modelname)

    transform = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std =[0.229, 0.224, 0.225])
                    ])
    
    dataset = torchvision.datasets.ImageNet(datasetpath, split='val', transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, pin_memory=True, num_workers=4)

    # Get the logits and targets
    dataset_logits = get_logits_targets(model, loader)

    # Save the dataset 
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    with open(fname, 'wb') as handle:
        pkl.dump(dataset_logits, handle, protocol=pkl.HIGHEST_PROTOCOL)

    return dataset_logits

    
def fix_randomness(seed=0):
    np.random.seed(seed=seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)

def get_imagenet_classes():
    df = pd.read_csv(dirname + '/map_clsloc.txt', delimiter=' ')
    arr = df['name'].to_numpy()
    return arr

def get_metrics_precomputed(est_labels,labels,losses,num_classes):
    labels = torch.nn.functional.one_hot(labels,num_classes)
    empirical_losses = (losses.view(1,-1) * (labels * (1-est_labels))).sum(dim=1)
    sizes = est_labels.sum(dim=1)
    return empirical_losses, sizes 


def platt_logits(calib_dataset, max_iters=10, lr=0.01, epsilon=0.01):
    calib_loader = torch.utils.data.DataLoader(calib_dataset, batch_size=1024, shuffle=False, pin_memory=True) 
    nll_criterion = nn.CrossEntropyLoss().cuda()

    T = nn.Parameter(torch.Tensor([1.3]).cuda())

    optimizer = optim.SGD([T], lr=lr)
    for iter in range(max_iters):
        T_old = T.item()
        for x, targets in calib_loader:
            optimizer.zero_grad()
            x = x.cuda()
            x.requires_grad = True
            out = x/T
            loss = nll_criterion(out, targets.long().cuda())
            loss.backward()
            optimizer.step()
        if abs(T_old - T.item()) < epsilon:
            break
    return T 

### Helping function for experiment

In [ ]:
def trial_precomputed(
    conformal_scores,
    raw_scores,
    alpha,
    epsilon,
    num_calib,
    seed,
    score_bins,
    gamma
):
    # Shuffle
    total = conformal_scores.shape[0]
    perm = torch.randperm(total)
    conformal_scores = conformal_scores[perm]
    raw_scores = raw_scores[perm]

    # Split calibration and validation sets
    calib_conformal_scores = 1 - conformal_scores[:num_calib]
    val_conformal_scores   = 1 - conformal_scores[num_calib:]
    calib_raw_scores       = 1 - raw_scores[:num_calib]
    val_raw_scores         = 1 - raw_scores[num_calib:]

    # ---- Private conformal thresholds ----
    shat = get_shat_from_scores_private(
        calib_conformal_scores, alpha, epsilon, gamma, score_bins
    )

    epsilon_conform = (epsilon**2)/2
    threshold_PrivQuant = PrivQuant(
        calib_conformal_scores, alpha, epsilon_conform, seed,
        lower_bound=0, upper_bound=1, delta=1e-16
    )

    q = 1 - alpha
    threshold_Lap_hist = dp_quantile_noisy_hist(
        calib_conformal_scores, q, epsilon, seed
    )

    # ---- Coverage (correctness) ----
    corrects            = (val_conformal_scores < shat)
    corrects_PrivQuant  = (val_conformal_scores < threshold_PrivQuant)
    corrects_Lap_hist   = (val_conformal_scores < threshold_Lap_hist)

    # ---- Set sizes ----
    sizes           = (val_raw_scores < shat).sum(dim=1)
    sizes_PrivQuant = (val_raw_scores < threshold_PrivQuant).sum(dim=1)
    sizes_Lap_hist  = (val_raw_scores < threshold_Lap_hist).sum(dim=1)

    return (
        corrects.float().mean().item(),
        corrects_PrivQuant.float().mean().item(),
        corrects_Lap_hist.float().mean().item(),
        sizes,
        sizes_PrivQuant,
        sizes_Lap_hist,
        shat,
        threshold_PrivQuant,
        threshold_Lap_hist
    )

### Experiment

In [ ]:
def experiment(alpha, epsilons, num_calib, seed, num_trials, imagenet_val_dir):
    # Clear old cache files
    cache_files = glob.glob(f'.cache/opt_{alpha}_*_dataframe_trial.pkl')
    for cache_file in cache_files:
        os.remove(cache_file)
    
    df_list = []

    for epsilon in epsilons:

        # Prepare cache directory
        os.makedirs('.cache', exist_ok=True)
        fname = f'.cache/opt_{alpha}_{epsilon}_{num_calib}_dataframe_trial.pkl'

        # Compute optimal bins + gamma
        mstar, gammastar = get_optimal_gamma_m(num_calib, alpha, epsilon)
        m = mstar
        gamma = gammastar
        score_bins = np.linspace(0, 1, m)

        # Expected column order
        expected_columns = [
            "$\\hat{s}$",
            "$\\hat{q}_$PCOQS",
            "thres_Lap_hist",
            "EXPONQ",
            "PCOQS",
            "Lap_hist",
            "sizes_EXPONQ",
            "sizes_PCOQS",
            "sizes_Lap_hist",
            "$\\alpha$",
            "$\\epsilon$"
        ]

        try:
            # Try loading cache
            df = pd.read_pickle(fname)

        except FileNotFoundError:
            
            # Compute dataset + trials
            dataset_precomputed = get_logits_dataset('ResNet152', 'Imagenet', imagenet_val_dir)
            print('Dataset loaded')

            classes_array = get_imagenet_classes()
            T = platt_logits(dataset_precomputed)

            logits, labels = dataset_precomputed.tensors
            scores = (logits / T.cpu()).softmax(dim=1)

            with torch.no_grad():
                conformal_scores = get_conformal_scores(scores, labels)

                local_df_list = []
                for i in tqdm(range(num_trials)):

                    trial_seed = seed + i

                    cvg1, cvg2, cvg3, szs1, szs2, szs3, shat, threshold_PrivQuant, threshold_Lap_hist = \
                        trial_precomputed(
                            conformal_scores, scores, alpha, epsilon,
                            num_calib, trial_seed, score_bins, gamma
                        )

                    # ---- FIXED SIZE FORMAT HERE ----
                    dict_local = {
                        "$\\hat{s}$": shat,
                        "$\\hat{q}_$PCOQS": threshold_PrivQuant,
                        "thres_Lap_hist": threshold_Lap_hist,

                        "EXPONQ": cvg1,
                        "PCOQS": cvg2,
                        "Lap_hist": cvg3,

                        # *** FIX: convert tensors -> list, then wrap in another list ***
                        "sizes_EXPONQ": [szs1.cpu().tolist()],
                        "sizes_PCOQS": [szs2.cpu().tolist()],
                        "sizes_Lap_hist": [szs3.cpu().tolist()],

                        "$\\alpha$": alpha,
                        "$\\epsilon$": epsilon
                    }

                    df_local = pd.DataFrame(dict_local)
                    local_df_list.append(df_local)

                df = pd.concat(local_df_list, axis=0, ignore_index=True)

                # Enforce column order
                df = df.reindex(columns=expected_columns)

                # Save cache
                df.to_pickle(fname)

        df_list.append(df)

    return df_list




if __name__ == "__main__":
    sns.set(palette='pastel', font='serif')
    sns.set_style('white')
    fix_randomness(seed=0)

    imagenet_val_dir = '' # TODO: put your imagenet data directory here

    alpha = 0.1
    epsilons = [0.05, 0.1, 0.5, 1, 3, 5]
    num_calib = 30000
    num_trials = 100
    seed = 123

    results = experiment(
        alpha, epsilons, num_calib,
        seed, num_trials,
        imagenet_val_dir=imagenet_val_dir
    )

### Save the results

In [ ]:
save_path = 'df_list_ImageNet_results_trial.pkl'
with open(save_path, 'wb') as f:
            pkl.dump(results, f)

### Processing and saving the results for plotting

In [ ]:
def flatten_size_column(col):
    """
    Extract size arrays when each cell is:
        [1, 3, 3, 5, ...]   (flat list)
    or occasionally wrapped like:
        [[1, 3, 3, 5, ...]]
    """
    flattened = []
    for cell in col:

        # Skip invalid entries
        if cell is None or (isinstance(cell, float) and np.isnan(cell)):
            continue

        inner = cell

        # If nested one extra level, unwrap
        if isinstance(inner, list) and len(inner) == 1 and isinstance(inner[0], list):
            inner = inner[0]

        # Now inner MUST be flat list
        if not isinstance(inner, (list, np.ndarray)):
            print("Warning: invalid size cell:", inner)
            continue

        flattened.append(np.array(inner, dtype=float))

    return flattened



def compute_trial_averages(size_series):
    """
    Compute mean set size for each trial.
    Input: size_series where each row is: [ [size1, size2, ..., sizeN] ]
    Output: array of size num_trials containing avg set size per trial.
    """
    flattened = flatten_size_column(size_series)

    if len(flattened) == 0:
        print("Warning: No valid size data")
        return np.array([])

    # All trials should have arrays of equal length
    lengths = set(arr.size for arr in flattened)
    if len(lengths) != 1:
        print("Warning: Inconsistent evaluation lengths across trials:", lengths)

    # Compute average size per trial
    return np.array([arr.mean() for arr in flattened])


def safe_to_dataframe(data_dict):
    """Convert dictionary to DataFrame even if lists have different lengths."""
    if not data_dict:
        return pd.DataFrame()
    
    max_len = max(len(v) for v in data_dict.values())
    padded = {
        k: np.pad(v, (0, max_len - len(v)),
                  mode="constant", constant_values=np.nan)
        for k, v in data_dict.items()
    }
    return pd.DataFrame(padded)


def main():
    # Load your experiment() output
    try:
        with open('df_list_ImageNet_results_trial.pkl', 'rb') as f:
            df_list = pkl.load(f)
    except FileNotFoundError:
        print("Error: Input file not found")
        return
    except pkl.PickleError:
        print("Error: Could not unpickle the file")
        return

    epsilons = [0.05, 0.1, 0.5, 1, 3, 5]

    results = {
        'coverage': {eps: {} for eps in epsilons},
        'avg_size': {eps: {} for eps in epsilons}
    }

    coverage_methods = ["EXPONQ", "PCOQS", "Lap_hist"]
    size_methods = ["EXPONQ", "PCOQS", "Lap_hist"]

    # ---- Parse each DataFrame ----
    for eps, df in zip(epsilons, df_list):
        try:
            # ---- COVERAGE ----
            for method in coverage_methods:
                if method in df.columns:
                    cov_data = df[method].dropna().astype(float).values
                    results['coverage'][eps][method] = cov_data

            # ---- SIZE ----
            for method in size_methods:
                size_key = f"sizes_{method}"
                if size_key in df.columns:
                    trial_avg_sizes = compute_trial_averages(df[size_key])

                    if len(trial_avg_sizes) > 0:
                        results['avg_size'][eps][method] = trial_avg_sizes
                    else:
                        print(f"Warning: No size averages for {method} at eps={eps}")

        except Exception as e:
            print(f"Error processing epsilon={eps}: {str(e)}")

    # ---- SAVE COVERAGE ----
    for eps in epsilons:
        cov_dict = results['coverage'][eps]
        if cov_dict:
            df_cov = safe_to_dataframe(cov_dict)
            if not df_cov.empty:
                df_cov.to_csv(f'coverage_epsilon_{eps}.csv', index=False)
            else:
                print(f"No coverage data for epsilon={eps}")

    # ---- SAVE AVERAGE SIZE ----
    for eps in epsilons:
        size_dict = results['avg_size'][eps]
        if size_dict:
            df_size = safe_to_dataframe(size_dict)
            if not df_size.empty:
                df_size.to_csv(f'avg_size_epsilon_{eps}.csv', index=False)
            else:
                print(f"No avg size data for epsilon={eps}")

    print("Processing complete. Files saved:")
    print("  coverage_epsilon_[eps].csv")
    print("  avg_size_epsilon_[eps].csv")


if __name__ == "__main__":
    main()
